> __Purpose:__ Implement an initial privacy attack to quantify how much re-identiifcation and linkability risk exists from personalization parameters (the decoder used in the co-adaptation algorithm). Thus, implement a basic ML model to link the decoder matrices back to the corresponding subject.  

-- 7 Subjects, therefore pure guessing would be 14.28% correct on average

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
import pickle
import random
from kcs_ml_infr import *

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model

from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier

# Make annoying KNN warning go away since I'm not going to edit scikit learn's code lol
import warnings
warnings.filterwarnings("ignore")

random.seed(a=1)

# Load In Data

In [6]:
keys = ['METACPHS_S106', 'METACPHS_S107','METACPHS_S108', 'METACPHS_S109', 'METACPHS_S110', 'METACPHS_S111', 'METACPHS_S112']

with open('Data\cphs_data_block1.pickle', 'rb') as handle:
    #refs_block1, poss_block1, dec_vels_block1, int_vel_block1, emgs_block1, Ws_block1, Hs_block1, alphas_block1, pDs_block1, times_block1, conditions_block1 = pickle.load(handle)
    _, _, _, _, _, Ws_block1, _, _, _, _, _ = pickle.load(handle)

with open('Data\cphs_data_block2.pickle', 'rb') as handle:
    #refs_block2, poss_block2, dec_vels_block2, int_vel_block2, emgs_block2, Ws_block2, Hs_block2, alphas_block2, pDs_block2, times_block2, conditions_block2 = pickle.load(handle)
    _, _, _, _, _, Ws_block2, _, _, _, _, _ = pickle.load(handle)
    
update_ix = np.load(r"Data\update_ix.npy")
print(update_ix.shape)
print(update_ix)

In [3]:
key_to_num = dict()
num_to_key = dict()
for idx, key in enumerate(keys):
    key_to_num[key] = idx
    num_to_key[idx] = key
    
# Params
# Set the number of k-folds
cv = 5
# Changed to 5 from 10 because the smallest class in cross val only has 7 instances
# NOTE: I also changed the default cv number in my function to be 5 because of this

my_metrics_cols = ['Algorithm', 'One Off Acc', 'CV Acc', 'K Folds']
#my_models = [LogisticRegression(), KNeighborsClassifier(), GradientBoostingClassifier()]
my_models = [LogisticRegression(), KNeighborsClassifier(), GaussianNB(), LinearSVC(), SGDClassifier(), DecisionTreeClassifier(), GradientBoostingClassifier()]

In [5]:
num_conds = 8
dec_flattened_df1 = pd.DataFrame(columns=["Subject", "Condition", "Update Number", "Flattened Decoder"])
dec_flattened_df2 = pd.DataFrame(columns=["Subject", "Condition", "Update Number", "Flattened Decoder"])

for key in keys:
    participant_dec1 = Ws_block1[key]
    participant_dec2 = Ws_block2[key]
    
    for my_cond in range(num_conds):
        for update_number, update_idx in enumerate(update_ix):
            dec_flattened_df1.loc[len(dec_flattened_df1)] = [key, my_cond, update_number, np.ravel(participant_dec1[my_cond, update_idx, :, :])]
            dec_flattened_df2.loc[len(dec_flattened_df2)] = [key, my_cond, update_number, np.ravel(participant_dec2[my_cond, update_idx, :, :])]
        
dec_flattened_df = pd.concat((dec_flattened_df1, dec_flattened_df2))

print(dec_flattened_df.shape)
dec_flattened_df.head()

(2128, 4)


,Subject,Condition,Update Number,Flattened Decoder
0,METACPHS_S106,0,0,"[0.002722144351611262, 0.002605931562722017, 0..."
1,METACPHS_S106,0,1,"[0.002722144351611262, 0.002605931562722017, 0..."
2,METACPHS_S106,0,2,"[-0.34157085409453486, 0.08410593293763585, -0..."
3,METACPHS_S106,0,3,"[-0.1738624752800762, 0.003705171262358347, 0...."
4,METACPHS_S106,0,4,"[0.8321061501817386, -1.3261242289666402, 0.28..."


In [ ]:
# SPLIT BASED ON CONDITION NUMBERS
# initialization conditions
pos_init = [0, 1, 4, 5]
neg_init = [2, 3, 6, 7]

In [ ]:
pos_dec_df = dec_flattened_df[dec_flattened_df["Condition"] in pos_init]
neg_dec_df = dec_flattened_df[dec_flattened_df["Condition"] in neg_init]

# Compare Accuracies Of First and Last Decoders

In [6]:
flat_dec_expanded_df = pd.DataFrame()
for my_row in range(dec_flattened_df.shape[0]):
    test=pd.DataFrame(dec_flattened_df.iloc[my_row,3]).T
    flat_dec_expanded_df = pd.concat((flat_dec_expanded_df, test))

flat_dec_expanded_df.reset_index(inplace=True, drop=True)
flat_dec_expanded_df.insert(loc=0, column='Update Number', value=list(dec_flattened_df['Update Number']))
flat_dec_expanded_df.insert(loc=0, column='Condition', value=list(dec_flattened_df['Condition']))
flat_dec_expanded_df.insert(loc=0, column='Subject', value=list(dec_flattened_df['Subject']))

print(flat_dec_expanded_df.shape)
flat_dec_expanded_df.head()

(2128, 131)


,Subject,Condition,Update Number,0,1,2,3,4,5,6,...,118,119,120,121,122,123,124,125,126,127
0,METACPHS_S106,0,0,0.002722,0.002606,0.003087,0.009610,0.009477,0.004051,0.000124,...,0.005588,0.005065,0.004476,0.004007,0.006816,0.005033,0.008904,0.005124,0.006046,0.005625
1,METACPHS_S106,0,1,0.002722,0.002606,0.003087,0.009610,0.009477,0.004051,0.000124,...,0.005588,0.005065,0.004476,0.004007,0.006816,0.005033,0.008904,0.005124,0.006046,0.005625
2,METACPHS_S106,0,2,-0.341571,0.084106,-0.540574,0.444320,-1.656320,-1.081761,1.806691,...,0.565411,0.128505,0.848800,-0.828621,-0.453219,0.081089,0.411674,0.070501,1.248486,-0.666425
3,METACPHS_S106,0,3,-0.173862,0.003705,0.407216,-0.515222,-0.217008,-0.418798,0.288675,...,0.144975,-0.436637,0.015547,-0.266492,-0.287574,-0.069196,-0.461702,-0.393759,0.979884,1.426735
4,METACPHS_S106,0,4,0.832106,-1.326124,0.287190,1.302253,-1.523494,-1.953003,1.082167,...,0.147737,-0.538394,0.088116,-0.280073,0.022182,-0.648191,1.269775,0.324295,0.327097,-2.468706


In [10]:
my_metrics_cols=['Algorithm', 'One Off Acc', 'CV Acc', 'K Folds']
key_to_num_dict=key_to_num

# Now Observe Effect of Decoder Update Number on Performance
## n=0

In [14]:
zero_df, zero_test_df = nth_decoder_model(flat_dec_expanded_df, 0, my_models, test=True)
zero_test_df.head(10)

,Algorithm,CV Acc,Test Acc,K Folds,N
0,LogisticRegression(),11.538,2.941,5,0
0,KNeighborsClassifier(),5.128,2.941,5,0
0,GaussianNB(),100.0,100.0,5,0
0,LinearSVC(),12.821,5.882,5,0
0,SGDClassifier(),10.256,14.706,5,0
0,DecisionTreeClassifier(),100.0,100.0,5,0
0,GradientBoostingClassifier(),100.0,100.0,5,0


## Big Loop

In [15]:
print(1)
one_df, one_test_df = nth_decoder_model(flat_dec_expanded_df, 1, my_models, test=True)
two_df, two_test_df = nth_decoder_model(flat_dec_expanded_df, 2, my_models, test=True)
three_df, three_test_df = nth_decoder_model(flat_dec_expanded_df, 3, my_models, test=True)
four_df, four_test_df = nth_decoder_model(flat_dec_expanded_df, 4, my_models, test=True)
five_df, five_test_df = nth_decoder_model(flat_dec_expanded_df, 5, my_models, test=True)
six_df, six_test_df = nth_decoder_model(flat_dec_expanded_df, 6, my_models, test=True)
print(7)
seven_df, seven_test_df = nth_decoder_model(flat_dec_expanded_df, 7, my_models, test=True)
eight_df, eight_test_df = nth_decoder_model(flat_dec_expanded_df, 8, my_models, test=True)
nine_df, nine_test_df = nth_decoder_model(flat_dec_expanded_df, 9, my_models, test=True)
ten_df, ten_test_df = nth_decoder_model(flat_dec_expanded_df, 10, my_models, test=True)
ele_df, ele_test_df = nth_decoder_model(flat_dec_expanded_df, 11, my_models, test=True)
print(12)
twe_df, twe_test_df = nth_decoder_model(flat_dec_expanded_df, 12, my_models, test=True)
thirt_df, thirt_test_df = nth_decoder_model(flat_dec_expanded_df, 13, my_models, test=True)
frtn_df, frtn_test_df = nth_decoder_model(flat_dec_expanded_df, 14, my_models, test=True)
fftn_df, fftn_test_df = nth_decoder_model(flat_dec_expanded_df, 15, my_models, test=True)
sixtn_df, sixtn_test_df = nth_decoder_model(flat_dec_expanded_df, 16, my_models, test=True)
svntn_df, svntn_test_df = nth_decoder_model(flat_dec_expanded_df, 17, my_models, test=True)
print('Complete')

1
7
12
Complete


## n=18

In [16]:
eightn_df, eightn_test_df = nth_decoder_model(flat_dec_expanded_df, 18, my_models, test=True)
eightn_test_df.head(10)

,Algorithm,CV Acc,Test Acc,K Folds,N
0,LogisticRegression(),69.231,70.588,5,18
0,KNeighborsClassifier(),35.897,55.882,5,18
0,GaussianNB(),67.949,64.706,5,18
0,LinearSVC(),70.513,79.412,5,18
0,SGDClassifier(),57.692,76.471,5,18
0,DecisionTreeClassifier(),33.333,47.059,5,18
0,GradientBoostingClassifier(),42.308,50.0,5,18


## Plot Accuracy As a Func of N

In [17]:
full_test_df = pd.concat((zero_test_df, one_test_df, two_test_df, three_test_df, four_test_df, 
           five_test_df, six_test_df, seven_test_df, eight_test_df, nine_test_df, 
           ten_test_df, ele_test_df, twe_test_df, thirt_test_df, frtn_test_df, 
           fftn_test_df, sixtn_test_df, svntn_test_df, eightn_test_df))

full_test_df.drop('K Folds', axis=1, inplace=True)
print(full_test_df.shape)

(133, 4)


In [18]:
lr_df = full_test_df[full_test_df['Algorithm']=='LogisticRegression()']
knn_df = full_test_df[full_test_df['Algorithm']=='KNeighborsClassifier()']
svc_df = full_test_df[full_test_df['Algorithm']=='LinearSVC()']
dt_df = full_test_df[full_test_df['Algorithm']=='DecisionTreeClassifier()']
gbt_df = full_test_df[full_test_df['Algorithm']=='GradientBoostingClassifier()']

In [38]:
plt.figure(figsize=(9,7))
plt.plot(list(range(19)), lr_df.iloc[:, 2], label='LogReg')
plt.plot(list(range(19)), knn_df.iloc[:, 2], label='KNN')
plt.plot(list(range(19)), svc_df.iloc[:, 2], label='SVC')
plt.plot(list(range(19)), dt_df.iloc[:, 2], label='DTree')
plt.plot(list(range(19)), gbt_df.iloc[:, 2], label='GBC')
plt.xticks(np.arange(0, 19, 1.0))
plt.yticks(np.arange(0, 120, 20.0))
plt.grid(axis='y')
plt.xlabel('Update Number')
plt.ylabel('Testing Accuracy')
plt.title('Model Accuracy as a function of Decoder Update Number')
plt.legend(loc='best')
plt.show()

NameError: name 'lr_df' is not defined

<Figure size 900x700 with 0 Axes>

In [39]:
my_x_updates = list(range(19))[2:]

plt.figure(figsize=(9,7))
plt.plot(my_x_updates, lr_df.iloc[2:, 2], label='LogReg')
plt.plot(my_x_updates, knn_df.iloc[2:, 2], label='KNN')
plt.plot(my_x_updates, svc_df.iloc[2:, 2], label='SVC')
plt.plot(my_x_updates, dt_df.iloc[2:, 2], label='DTree')
plt.plot(my_x_updates, gbt_df.iloc[2:, 2], label='GBC')
plt.xticks(np.arange(0, 19, 1.0))
plt.yticks(np.arange(0, 120, 20.0))
plt.grid(axis='y')
plt.xlabel('Update Number')
plt.ylabel('Testing Accuracy')
plt.title('Model Accuracy as a function of Decoder Update Number')
plt.legend(loc='best')
plt.show()

NameError: name 'lr_df' is not defined

<Figure size 900x700 with 0 Axes>